# Merged model of photosynthesis
###### This is a Jupyter notebook for the model submitted by Matuszynska et al 2018 in Physiologia Plantarum.


## 1. Package import and initial conditions
###### The first cell imports all necessary packages and defines dictionaries of initial conditions for bistability analysis and all other simulations. The instantiation of the model in modelbase is performed via the function "instantiate()" in the file "instantiate.py". The construction of the ODEs is performed by the file "model.py", the kinetic equations are defined in the file "reactionrates". The parameters are defined in the file "parameters".

# IMPORTANT
###### The first cell needs to be runned in order to perform simulations in any other cell


In [3]:
import modelbase
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import instantiate
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
p, rat, m, s = instantiate.instantiate()

#Regular initial values are selected for non-stressed conditions.
#Half Pools -> non stress conditions

init = {
        "PQ":p.PQtot/2,
        "PC":p.PCtot/2,
        "Fd":p.Fdtot/2,
        "ATP":0.12,
        "NADPH":0.281543418344,
        "H (lumen)":rat.pHinv(7.2),
        "LHC":0.9,
        "Psbs":0.9,
        "Vx":0.9,
        "PGA":2.15,
        'BPGA':0.000907499521924,
        'GAP':0.011839616887,
        'DHAP':0.260471552645,
        'FBP':0.11,
        'F6P':0.36,
        'G6P':0.46,
        'G1P':0.166166918189,
        'SBP':0.2,
        'S7P':0.56,
        'E4P':0.0330766864679,
        'X5P':0.0374527459593,
        'R5P':0.0627333486958,
        'RUBP':0.45,
        'RU5P':0.02,
        'Fluo':0.2,
        "Light":m.par.PFD}




Could not find GLIMDA


Created a virtual organism. Experiment ready to run


## 2. Light-Dark-Light simulations

##### The following cell includes a function that simulates the model in a LDL light protocol. This function (LDL) takes the simulate object, initial conditions, light intensities of the light phases, and duration of the darkphases as arguments. The argument Tflash determines the step size of the piecewise simulation.

The following cell defines a function which includes a piecewise simulation of the model. The purpose of the piecewise simulation is to change the light intensity in defined timesteps.

In [1]:
%%capture
import numpy as np
def LDL(s, y0, Tmax,intensity,darkphase, Tflash=5.):  
    '''Tflash defines the quasi step size of the piecewise simulation'''
    
    s.set_initial_value(y0, t0=0)
    t = 0
    y = y0
    
    s.model.par.update({'PFD':intensity})
    
    s.set_initial_value(y0, t0=0)
    try:
        '''if Assimulo present'''
        s.integrator.atol = 1.e-14
    except:
        pass
    t = 0
    y = y0

    while s.successful() and t < Tmax:

        if t>=300 and t<(300+darkphase):
            
            s.model.par.update({'PFD': 5.})
        else:
            s.model.par.update({'PFD': intensity})

        tfl = np.linspace(t, t+Tflash, 100)
        
        s.timeCourse(tfl, y)
        t = s.getT()[-1]
        y = s.getVarsByName(s.model.cpdNames)[-1]
        
    return s, s.model



The following cell calculates and plots the results from Figure 2 of the publication.

In [ ]:
y0 = np.array(list(init.values()))
PhosphoMAX=15
fig = plt.figure()
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)

s = modelbase.Simulator(m)
s, m = LDL(s, y0, Tmax=1200,intensity=20.,darkphase=200.,Tflash=100.)
t1=s.getT()

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex1=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub1=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='royalblue',linestyle=':',linewidth=4.,label='20 PFD',zorder=10.)
ax2.plot(t1,yvex1,color='royalblue',linestyle='--',linewidth=4.,label='20 PFD',zorder=10.)
ax3.plot(t1,yrub1,color='royalblue',linestyle='-.',linewidth=4.,label='20 PFD',zorder=10.)
sX1 = modelbase.Simulator(m)
s, m = LDL(sX1, y0, Tmax=1200,intensity=50.,darkphase=200.,Tflash=100.)
t1=s.getT()

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='forestgreen',linestyle=':',linewidth=4.,label='50 PFD',zorder=8.)
ax2.plot(t1,yvex2,color='forestgreen',linestyle='--',linewidth=4.,label='50 PFD',zorder=8.)
ax3.plot(t1,yrub2,color='forestgreen',linestyle='-.',linewidth=4.,label='50 PFD',zorder=8.)

sX1 = modelbase.Simulator(m)
s, m = LDL(sX1, y0, Tmax=1200,intensity=100.,darkphase=200.,Tflash=100.)
t1=s.getT()

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='indianred',linestyle=':',linewidth=4.,label='100 PFD',zorder=6.)
ax2.plot(t1,yvex2,color='indianred',linestyle='--',linewidth=4.,label='100 PFD',zorder=6.)
ax3.plot(t1,yrub2,color='indianred',linestyle='-.',linewidth=4.,label='100 PFD',zorder=6.)

sX1 = modelbase.Simulator(m)
s, m = LDL(sX1, y0, Tmax=1200,intensity=150.,darkphase=200.,Tflash=100.)
t1=s.getT()

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='darkorange',linestyle=':',linewidth=4.,label='150 PFD',zorder=4.)
ax2.plot(t1,yvex2,color='darkorange',linestyle='--',linewidth=4.,label='150 PFD',zorder=4.)
ax3.plot(t1,yrub2,color='darkorange',linestyle='-.',linewidth=4.,label='150 PFD',zorder=4.)

sX1 = modelbase.Simulator(m)
s, m = LDL(sX1, y0, Tmax=1200,intensity=200.,darkphase=200.,Tflash=100.)#
t1=s.getT()
y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='k',linestyle=':',linewidth=4.,label='200 PFD',zorder=2.)
ax2.plot(t1,yvex2,color='k',linestyle='--',linewidth=4.,label='200 PFD',zorder=2.)
ax3.plot(t1,yrub2,color='k',linestyle='-.',linewidth=4.,label='200 PFD',zorder=2.)

ax1.axvspan(300,500,alpha=0.4,color='dimgray')
ax2.axvspan(300,500,alpha=0.4,color='dimgray')
ax3.axvspan(300,500,alpha=0.4,color='dimgray')

ax1.set_facecolor('white')
ax1.grid(color='lightgrey')

ax2.set_facecolor('white')
ax2.grid(color='lightgrey')

ax3.set_facecolor('white')
ax3.grid(color='lightgrey')

ax1.set_ylabel('[Int. free phosphate] (mM)',fontsize=14.)
ax2.set_ylabel('TPT translocators (mM/s)',fontsize=14.)
ax3.set_ylabel('RuBisCO (mM/s)',fontsize=14.)

ax1.text(245,1.07, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax1.text(390, 1.07, 'D', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax1.text(590, 1.07, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax1.set_ylim((0.4,1.15))


ax2.text(245,.25, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax2.text(390, .25, 'D', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax2.text(590, .25, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})

ax3.text(245,1., 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax3.text(390, 1., 'D', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax3.text(590, 1., 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})

ax2.legend(loc='best',bbox_to_anchor=(1., 0.7), prop={'size': 16})

ax1.tick_params(axis='both', which='major', labelsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax3.tick_params(axis='both', which='major', labelsize=18)

ax3.set_xlabel('Time (s)',fontsize=20.)

ax1.set_xlim((200,850))
ax2.set_xlim((200,850))
ax3.set_xlim((200,850))

plt.show()


## 3. Bistability analysis

### The following cell includes time course simulations with an alternative set of initial conditions (init_RU5P). In this set of initial conditions, all CBB cycle intermediate concentrations are 0 except for RU5P. In the time course simulations, the initial RU5P concentration is systematically increased. 

The following cell defines an alternative set of initial conditions and simulates the model over time with different initial concentrations of RU5P.

In [ ]:
%%capture
init_RU5P = {
        "PQ":p.PQtot/2,
        "PC":p.PCtot/2,
        "Fd":p.Fdtot/2,
        "ATP":0.12,
        "NADPH":0.281543418344,
        "H (lumen)":rat.pHinv(7.2),
        "LHC":0.9,
        "Psbs":0.9,
        "Vx":0.9,
        "PGA":0.,
        'BPGA':0.0,
        'GAP':0.0,
        'DHAP':0.,
        'FBP':0.,
        'F6P':0.,
        'G6P':0.,
        'G1P':0.,
        'SBP':0.,
        'S7P':0.,
        'E4P':0.,
        'X5P':0.,
        'R5P':0.,
        'RUBP':0.,
        'RU5P':.2,
        'Fluo':0.2,
        "Light":m.par.PFD}


init_ru5p=np.round(np.linspace(0.35,0.5,6),3)
rubspeicher=[]
res_list_atp=[]
res_list_starch=[]
res_list_vex=[]
    
fig = plt.figure()
ax1 = fig.add_subplot(111)

for i in init_ru5p:
    init_RU5P['RU5P']=i
    print(init_RU5P)
    y0 = np.array(list(init_RU5P.values()))
    s = modelbase.Simulator(m)
    s.set_initial_value(y0, t0=0)
    s.integrator.atol = 1.e-16
    s.model.par.update({'PFD':500})
    T=np.linspace(0.,600,300)
    s.timeCourse(T,y0)
    t=s.getT()
    res=s.getVarByName('RU5P')

    ax1.plot(t,res,linewidth=4.,label=str(i)+' mM init. [RU5P]')



The following cell is plotting the results for Figure 3 of the publication.

In [ ]:
ax1.plot(t,np.full(len(t),0.00248),color='k',linewidth=3.,linestyle=':',label='Critical concentration')    
ax1.set_ylabel('[RU5P] (mM)',fontsize=20.)
ax1.set_facecolor('white')
ax1.grid(color='lightgrey')
ax1.tick_params(axis='both', which='major', labelsize=17)
ax1.set_xlabel('Time (s)',fontsize=20.)
ax1.legend(loc='best', prop={'size': 16})
ax1.set_xlim((7,600))
ax1.set_ylim((0.,0.025))

plt.show()

## 4. Investigating RU5P resupply

### The following cell includes simulations to steady state with the additional reaction $V_{\mathrm{oxPPP}}$, which constantly introduces RU5P into the system. In the simulation, the kinetic parameter  $K_{\mathrm{oxPPP}}$ is systematically increased.

### The reaction $V_{\mathrm{oxPPP}}$ is only intact in this simulation.

The following cell calculates the model and systematically changes the parameter 'oxPPP'

In [ ]:
T=np.linspace(0.,5e7)
oxPPP_list=np.linspace(0,0.075,100)

res_list=[]
res_list_ru5p=[]
res_list_H=[]
res_list_nadph=[]

for i in oxPPP_list:
    
    y0 = np.array(list(init.values()))
    s = modelbase.Simulator(m)
    s.set_initial_value(y0, t0=0)
    s.integrator.atol = 1.e-14

    s.model.par.update({'PFD':5})
    s.model.par.update({'oxPPP':i})
    s.timeCourse(T,y0)

    atpCC=s.getVarByName('ATP')
    res_list.append((atpCC[-1]/p.APtot))
    ru5pCC=s.getVarByName('RU5P')
    res_list_ru5p.append(ru5pCC[-1])
    protonsOX=s.getVarByName('H')
    res_list_H.append(rat.pH(protonsOX[-1]))
    nadphox=s.getVarByName('NADPH')[-1]
    res_list_nadph.append(nadphox/p.NADPtot)    


The following cell plots the results for Figure 4 of the publication.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(311)
ax1.plot(oxPPP_list,res_list,color='royalblue',linewidth=4.)
ax1.set_ylabel(r'$\frac{[ATP]}{[ATP]+[ADP]}$',fontsize=20.)
ax1.set_facecolor('white')
ax1.grid(color='lightgrey')
ax1.tick_params(axis='both', which='major', labelsize=15)


ax2 = fig.add_subplot(312)
ax2.plot(oxPPP_list,res_list_ru5p,color='forestgreen',linewidth=4.)
ax2.plot(oxPPP_list,np.full(len(oxPPP_list),0.00248),color='k',linewidth=2.,linestyle=':',label='Critical Concentration')
ax2.legend(loc='best', prop={'size': 15})
ax2.set_ylabel('[RU5P] (mM)',fontsize=20.)
ax2.set_facecolor('white')
ax2.grid(color='lightgrey')
ax3 = fig.add_subplot(313)
ax3.plot(oxPPP_list,res_list_H,color='indianred',linewidth=4.)

ax3.set_ylabel('Lumenal pH',fontsize=20.)
ax3.set_xlabel('RU5P influx in mM/s',fontsize=20.)
ax3.set_facecolor('white')
ax3.grid(color='lightgrey')

ax2.tick_params(axis='both', which='major', labelsize=15)
ax3.tick_params(axis='both', which='major', labelsize=15)

plt.show();

## 5. Steady state behaviour in varying light intensities and CBB cycle velocities.

### The following cell includes simulations to steady state in different light intensities and CBB cycle intensities. The $V_{\mathrm{max}}$ values of selected CBB cycle reactions are multiplied with a systematically increased value ($f_{\mathrm{CBB}}$), as well as the light intensity. The function gives the option of visualizing already calculated results, or recalculate the results (which takes quite some time).

### The function *Surface* takes one argument, and another optional argument:
### The first argument determines the output of the simulation. The user can choose from the strings "pH",  "ATP", "Starch" and "TPT", to visualizes simulation results with one of the four mentioned outputs.
### The second argument is optional and determines if the function visualizes already calculates results (the argument "file") or if the function calculates the results from scratch (the argument "calculate"). The calculation takes approximately one hour.

In [5]:
def Surface(output,option='file'):
    if option=='file':
        PFDs=[1.,2.,5.,8.,10.,20.,30.,40.,50.,60.,70.,80.,90.,100.,200.,300.,400.,500.,600.,700]
        Xs=np.round(np.linspace(1.,10.,10),1)
        PFDs=np.array(PFDs)

        atparray=np.load('Surface_results/ATP_surface.npy')
        starcharray=np.load('Surface_results/Starch_surface.npy')
        vex_array=np.load('Surface_results/TPT_surface.npy')
        proton_array=np.load('Surface_results/Proton_surface.npy')
        proton_array=rat.pH(proton_array)

        hf = plt.figure()
        ha = hf.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(PFDs, Xs) 
        output_dict={'ATP':atparray,'Starch':starcharray,'TPT':vex_array,'pH':proton_array}
        
        Z=output_dict[output]

        Gx, Gy = np.gradient(Z) 
        G = (Gx**2+Gy**2)**.5  
        N = G/G.max()  

        surf=ha.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=plt.cm.RdYlBu,linewidth=1, antialiased=True,alpha=1.)

        cb=hf.colorbar(surf)
        cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=18)

        ha.set_xlabel('Light', labelpad=20.2,fontsize=22.)
        ha.set_ylabel(r'$f_\mathrm{CBB}$', labelpad=20.2,fontsize=22.)
        ha.tick_params(labelsize=18)
        ha.set_facecolor('white')

        ha.set_zlabel(output, labelpad=21.2,fontsize=22.,rotation=90)
        ha.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ha.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ha.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

        ha.zaxis.grid('true',color='b')
        plt.show()
    elif option=='calculate':
        y0 = np.array(list(init.values()))
        s = modelbase.Simulator(m)
        s.set_initial_value(y0, t0=0)
        s.integrator.atol = 1.e-14

        T=np.linspace(0.,5e10)
        PFDs=[1.,2.,5.,8.,10.,20.,30.,40.,50.,60.,70.,80.,90.,100.,200.,300.,400.,500.,600.,700]
        Xs=np.round(np.linspace(1.,10.,10),1)

        speicherATP=[]
        speicherstarch=[]
        SPEICHER_H=[]
        speicher_vex=[]
        s.model.par.update({'kcyc' : 1.})#3.
        for X in Xs:
            speicherX=[]
            speich_starch=[]

            speich_H=[]
            res_list_vex=[]
            s.model.par.update({'V1' : 0.34*8*X})
            s.model.par.update({'V6' : 0.2*8*X})
            s.model.par.update({'V9' : 0.04*8*X})
            s.model.par.update({'V13' : 0.9999*8*X})
            s.model.par.update({'Vst' : 0.04*8*X})

            for i in PFDs:
                s.model.par.update({'PFD':i})
                print(s.model.par.PFD)
                s.timeCourse(T, y0)
                y=s.getY()
                atp=s.getVarByName('ATP')
                starch=s.getRate('vStarch')
                h=s.getVarByName('H')

                speicherX.append(atp[-1])
                speich_starch.append(starch[-1])
                speich_H.append(h[-1])
                VEX=(s.getRate('vpga')[-1]+s.getRate('vgap')[-1]+s.getRate('vDHAP')[-1])
                res_list_vex.append(VEX)  

            speicherATP.append(speicherX)
            speicherstarch.append(speich_starch)
            SPEICHER_H.append(speich_H)
            speicher_vex.append(res_list_vex)

        speicher_vex=np.array(speicher_vex)
        speicherATP=np.array(speicherATP)
        speicherATPnorm=speicherATP/2.5

        speicherRUBISCO=np.array(speicherRUBISCO)
        
        SPEICHER_H=np.array(SPEICHER_H)
        SPEICHER_H=rat.pH(SPEICHER_H)
        
        np.save('ATP_surface',speicherATPnorm)
        np.save('Starch_surface',speicherstarch)
        np.save('Proton_surface',SPEICHER_H)
        np.save('TPT_surface',speicher_vex)
        hf = plt.figure()
        ha = hf.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(PFDs, Xs) 
        
        output_dict={'ATP':speicherATPnorm,'Starch':speicherstarch,'TPT':speicher_vex,'pH':SPEICHER_H}
        
        Z=output_dict[output]

        Gx, Gy = np.gradient(Z) 
        G = (Gx**2+Gy**2)**.5  
        N = G/G.max()  

        surf=ha.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=plt.cm.RdYlBu,linewidth=1, antialiased=True,alpha=1.)

        cb=hf.colorbar(surf)
        cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=18)

        ha.set_xlabel('Light', labelpad=20.2,fontsize=22.)
        ha.set_ylabel(r'$f_\mathrm{CBB}$', labelpad=20.2,fontsize=22.)
        ha.tick_params(labelsize=18)
        ha.set_facecolor('white')

        ha.set_zlabel(output, labelpad=21.2,fontsize=22.,rotation=90)
        ha.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ha.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ha.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

        ha.zaxis.grid('true',color='b')
        plt.show()

Surface('ATP')

## 5. Defining a function for flux control coefficients

### The following cell includes a function which calculates all flux control coefficients of one $V_{\mathrm{max}}$. The function also includes optional arguments like the $f_{\mathrm{CBB}}$ constant, light intensity, $k_{\mathrm{cyc}}$ parameter and variation.

In [4]:
def response_coefficient(par,CBBspeed=1.,lightintensity=100.,kcyc=1., var=0.01):
    or_dic=m.par.__dict__
    
    T=np.linspace(0.,5e15)
    y0 = np.array(list(init.values()))
    s = modelbase.Simulator(m)
    
    s.model.par.update({'V1' : 0.34*8*CBBspeed})
    s.model.par.update({'V6' : 0.2*8*CBBspeed})
    s.model.par.update({'V9' : 0.04*8*CBBspeed})
    s.model.par.update({'V13' : 0.9999*8*CBBspeed})
    s.model.par.update({'Vst' : 0.04*8*CBBspeed})
    s.model.par.update({'kcyc' : kcyc})
    
    s.model.par.update({'PFD':lightintensity})
    Y=dict()
    X=dict()
    s.timeCourse(T,y0)
    for name in m.rateNames():
        X.update({name:s.getRate(name)[-1]})
    for name in m.cpdNames:
        Y.update({name:s.getVarByName(name)[-1]})
    Y1=dict()
    X1=dict()
    xvar=or_dic[par]
    s.model.par.update({par:xvar*(1+var)})

    s.timeCourse(T,y0)
    for name in m.rateNames():
        X1.update({name:s.getRate(name)[-1]})
    for name in m.cpdNames:
        Y1.update({name:s.getVarByName(name)[-1]})
    Y99=dict()
    X99=dict()
    s.model.par.update({par:xvar*(1-var)})

    s.timeCourse(T,y0)    
    for name in m.rateNames():
        X99.update({name:s.getRate(name)[-1]})
    for name in m.cpdNames:
        Y99.update({name:s.getVarByName(name)[-1]})
    flux_coefficient_dict=dict()
    concentration_coefficient_dict=dict()
    
    
    
    for name in m.rateNames():
        if name != 'voxPPP':
            fluxcoefficient= ( ( ( X1[name] - X99[name] ) / X[name] ) )/ (var*2)
            flux_coefficient_dict.update({name:fluxcoefficient})
    for name in m.cpdNames:
        conc_coefficient= ( ( ( Y1[name] - Y99[name] ) / Y[name] ) )/ (var*2)
        concentration_coefficient_dict.update({name:conc_coefficient})
    return [flux_coefficient_dict, concentration_coefficient_dict]



## 6. Calculating flux control coefficients

### The following cell calculates flux control coefficients for different light intensities and  $f_{\mathrm{CBB}}$ values.
### It defines the function *FluxControlCoefficients* which calculates all flux control coefficients for all $V_{\mathrm{max}}$ values in the model for all optional light intensities and  $f_{\mathrm{CBB}}$ values.
### The second function, *SupplyDemand* sorts and calculates the absolute overall control of all demand and supply reactions, and visualizes the normalized overall demand control. It provides the option of visualizing already calculated results (default argument "file") or to recalculate the results (argument "calculate"). The recalculation takes approximately 30 minutes.

In [12]:

import pickle
cbblist=np.round(np.linspace(1.,3,3),0)
PFDlist=np.round(np.linspace(50.,600,12),0) 

Vmax_list=['PSItot','PSIItot','kPTOX','kCytb6f','kcyc','kNDH','kcatFNR','kLeak','kStt7','kPph1','kATPsynth',
           'kProtonationL','kDeprotonation','kDeepoxV','kEpoxZ',

           'V1','k','V6','V9','V13','Vx','Vst']

demand_pars=['V1' ,'V6' ,'V9', 'V13','Vst', 'Vx'] 
def FluxControlCoefficients(PFDrange=np.round(np.linspace(50.,600,12),0),CBBrange=np.round(np.linspace(1.,3,3),0)):
    cbblist=CBBrange
    PFDlist=PFDrange

    for CBB in cbblist:
        for pfd in PFDlist:

            FluxControlCoefficients=dict()
            ConcentrationControlCoefficients=dict()
            cnt=0
            for vmax in Vmax_list:
                cnt+=1
                print(str(cnt)+' of '+str(len(Vmax_list)))
                RC=response_coefficient(vmax,lightintensity=pfd,CBBspeed=CBB)
                FluxControlCoefficients.update({vmax:RC[0]})
                ConcentrationControlCoefficients.update({vmax:RC[1]})


            pickle.dump( FluxControlCoefficients, open( "flux-pfd"+str(int(pfd))+"cbb"+str(int(CBB))+".p", "wb" ) )

        
        

def SupplyDemand(option='file',PFDrange=None, CBBrange=None):
    if option=='file':
        DemandSupply=[]
        for CBB in cbblist:
            print(CBB)


            DemSup=[]
            for pfd in PFDlist:

                d1=[]
                s1=[]
                FCC = pickle.load( open( "FCC/flux-pfd"+str(int(pfd))+"cbb"+str(int(CBB))+".p", "rb" ) )
                for rate in FCC['V9']:
                    for par in Vmax_list:
                        if par in demand_pars:
                            d1.append(abs(FCC[par][rate]))
                        else:
                            s1.append(abs(FCC[par][rate]))


                DemSup.append(sum(d1)/(sum(d1)+sum(s1)))

            DemandSupply.append(DemSup)

        DemandSupply=np.array(DemandSupply)
        fig, axis = plt.subplots() 
        heatmap = axis.pcolor(DemandSupply, cmap=plt.cm.RdYlBu_r)

        axis.set_yticks(np.arange(DemandSupply.shape[0])+0.5, minor=False)
        axis.set_xticks(np.arange(DemandSupply.shape[1])+0.5, minor=False)

        axis.set_yticklabels(cbblist, minor=False)
        axis.set_xticklabels(PFDlist, minor=False)

        axis.set_xlabel('Light',fontsize=20.)
        axis.set_ylabel(r'$f_\mathrm{CBB}$',fontsize=20.)

        axis.tick_params(labelsize=15)
        axis.text(0.6,2.6,r'$C_{Supply} > C_{Demand}$',fontsize=30.,color='w')
        axis.text(8.1,0.35,r'$C_{Demand} > C_{Supply}$',fontsize=30.,color='w')

        plt.xticks(rotation=90)

        cb=fig.colorbar(heatmap)
        cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=18)

        plt.tight_layout() 
    elif option=='calculate':
        FluxControlCoefficients()
        DemandSupply=[]
        for CBB in cbblist:
            print(CBB)


            DemSup=[]
            for pfd in PFDlist:

                d1=[]
                s1=[]
                FCC = pickle.load( open( "flux-pfd"+str(int(pfd))+"cbb"+str(int(CBB))+".p", "rb" ) )
                for rate in FCC['V9']:
                    for par in Vmax_list:
                        if par in demand_pars:
                            d1.append(abs(FCC[par][rate]))
                        else:
                            s1.append(abs(FCC[par][rate]))


                DemSup.append(sum(d1)/(sum(d1)+sum(s1)))

            DemandSupply.append(DemSup)

        DemandSupply=np.array(DemandSupply)
        fig, axis = plt.subplots() 
        heatmap = axis.pcolor(DemandSupply, cmap=plt.cm.RdYlBu_r)

        axis.set_yticks(np.arange(DemandSupply.shape[0])+0.5, minor=False)
        axis.set_xticks(np.arange(DemandSupply.shape[1])+0.5, minor=False)

        axis.set_yticklabels(cbblist, minor=False)
        axis.set_xticklabels(PFDlist, minor=False)

        axis.set_xlabel('Light',fontsize=20.)
        axis.set_ylabel(r'$f_\mathrm{CBB}$',fontsize=20.)

        axis.tick_params(labelsize=15)
        axis.text(0.6,2.6,r'$C_{Supply} > C_{Demand}$',fontsize=30.,color='w')
        axis.text(8.1,0.35,r'$C_{Demand} > C_{Supply}$',fontsize=30.,color='w')

        plt.xticks(rotation=90)

        cb=fig.colorbar(heatmap)
        cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=18)

        plt.tight_layout() 
SupplyDemand()

1.0
2.0
3.0


# OPTIONAL CODE
The following cells include code from figures that are in the supplementary material and for curiosity

The cell below calculates the control coefficients of RuBisCO for different fCBBs and light intensities.

In [13]:

import pickle
cbblist=np.round(np.linspace(1.,3,3),0)
PFDlist=np.round(np.linspace(50.,600,12),0) 

Vmax_list=['PSItot','PSIItot','kPTOX','kCytb6f','kcyc','kNDH','kcatFNR','kLeak','kStt7','kPph1','kATPsynth',
           'kProtonationL','kDeprotonation','kDeepoxV','kEpoxZ',

           'V1','k','V6','V9','V13','Vx','Vst']

demand_pars=['V1' ,'V6' ,'V9', 'V13','Vst', 'Vx'] 
def FluxControlCoefficients(PFDrange=np.round(np.linspace(50.,600,12),0),CBBrange=np.round(np.linspace(1.,3,3),0)):
    cbblist=CBBrange
    PFDlist=PFDrange

    for CBB in cbblist:
        for pfd in PFDlist:

            FluxControlCoefficients=dict()
            ConcentrationControlCoefficients=dict()
            cnt=0
            for vmax in Vmax_list:
                cnt+=1
                print(str(cnt)+' of '+str(len(Vmax_list)))
                RC=response_coefficient(vmax,lightintensity=pfd,CBBspeed=CBB)
                FluxControlCoefficients.update({vmax:RC[0]})
                ConcentrationControlCoefficients.update({vmax:RC[1]})


            pickle.dump( FluxControlCoefficients, open( "flux-pfd"+str(int(pfd))+"cbb"+str(int(CBB))+".p", "wb" ) )

        
        

def SupplyDemandVIP(choice,option='file',PFDrange=None, CBBrange=None):
    if option=='file':
        DemandSupply=[]
        VIP=[]
        for CBB in cbblist:
            print(CBB)


            DemSup=[]
            special=[]
            for pfd in PFDlist:

                d1=[]
                s1=[]
                spc=[]
                FCC = pickle.load( open( "FCC/flux-pfd"+str(int(pfd))+"cbb"+str(int(CBB))+".p", "rb" ) )




                special.append(FCC[choice]['vRuBisCO'])

            VIP.append(special)
        VIP=np.array(VIP)
        print(np.shape(VIP))
        fig, axis = plt.subplots() 
        heatmap = axis.pcolor(VIP, cmap=plt.cm.RdYlBu_r)

        axis.set_yticks(np.arange(VIP.shape[0])+0.5, minor=False)
        axis.set_xticks(np.arange(VIP.shape[1])+0.5, minor=False)

        axis.set_yticklabels(cbblist, minor=False)
        axis.set_xticklabels(PFDlist, minor=False)

        axis.set_xlabel('Light',fontsize=20.)
        axis.set_ylabel(r'$f_\mathrm{CBB}$',fontsize=20.)

        axis.tick_params(labelsize=15)

        
        plt.title(str(choice))
        plt.xticks(rotation=90)

        cb=fig.colorbar(heatmap)
        cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=18)

        plt.tight_layout() 

SupplyDemandVIP('V1')

1.0
2.0
3.0
(3, 12)


The cell below calculates the control coefficients of the Seduheptulose-1,7-Biphosphatase for different fCBBs and light intensities.

In [46]:

import pickle
cbblist=np.round(np.linspace(1.,3,3),0)
PFDlist=np.round(np.linspace(50.,600,12),0) 

Vmax_list=['PSItot','PSIItot','kPTOX','kCytb6f','kcyc','kNDH','kcatFNR','kLeak','kStt7','kPph1','kATPsynth',
           'kProtonationL','kDeprotonation','kDeepoxV','kEpoxZ',

           'V1','k','V6','V9','V13','Vx','Vst']

demand_pars=['V1' ,'V6' ,'V9', 'V13','Vst', 'Vx'] 
def FluxControlCoefficients(PFDrange=np.round(np.linspace(50.,600,12),0),CBBrange=np.round(np.linspace(1.,3,3),0)):
    cbblist=CBBrange
    PFDlist=PFDrange

    for CBB in cbblist:
        for pfd in PFDlist:

            FluxControlCoefficients=dict()
            ConcentrationControlCoefficients=dict()
            cnt=0
            for vmax in Vmax_list:
                cnt+=1
                print(str(cnt)+' of '+str(len(Vmax_list)))
                RC=response_coefficient(vmax,lightintensity=pfd,CBBspeed=CBB)
                FluxControlCoefficients.update({vmax:RC[0]})
                ConcentrationControlCoefficients.update({vmax:RC[1]})


            pickle.dump( FluxControlCoefficients, open( "flux-pfd"+str(int(pfd))+"cbb"+str(int(CBB))+".p", "wb" ) )

        
        

def SupplyDemandVIP(choice,option='file',PFDrange=None, CBBrange=None):
    if option=='file':
        DemandSupply=[]
        VIP=[]
        for CBB in cbblist:
            print(CBB)


            DemSup=[]
            special=[]
            for pfd in PFDlist:

                d1=[]
                s1=[]
                spc=[]
                FCC = pickle.load( open( "FCC/flux-pfd"+str(int(pfd))+"cbb"+str(int(CBB))+".p", "rb" ) )
                for rate in FCC['V9']:
                    spc.append(abs(FCC[choice][rate]))



                special.append(sum(spc))

            VIP.append(special)
        VIP=np.array(VIP)
        print(np.shape(VIP))
        fig, axis = plt.subplots() 
        heatmap = axis.pcolor(VIP, cmap=plt.cm.RdYlBu_r)

        axis.set_yticks(np.arange(VIP.shape[0])+0.5, minor=False)
        axis.set_xticks(np.arange(VIP.shape[1])+0.5, minor=False)

        axis.set_yticklabels(cbblist, minor=False)
        axis.set_xticklabels(PFDlist, minor=False)

        axis.set_xlabel('Light',fontsize=20.)
        axis.set_ylabel(r'$f_\mathrm{CBB}$',fontsize=20.)

        axis.tick_params(labelsize=15)

        plt.title(str(choice))
        plt.xticks(rotation=90)

        cb=fig.colorbar(heatmap)
        cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=18)

        plt.tight_layout() 

SupplyDemandVIP('V9')

1.0
2.0
3.0
(3, 12)


The following cell performs a time course simulation using the alternative initial concentration set, but in different light intensities than similar simulations in the code above.

In [19]:
%%capture
init_RU5P = {
        "PQ":p.PQtot/2,
        "PC":p.PCtot/2,
        "Fd":p.Fdtot/2,
        "ATP":0.12,
        "NADPH":0.281543418344,
        "H (lumen)":rat.pHinv(7.2),
        "LHC":0.9,
        "Psbs":0.9,
        "Vx":0.9,
        "PGA":0.,
        'BPGA':0.0,
        'GAP':0.0,
        'DHAP':0.,
        'FBP':0.,
        'F6P':0.,
        'G6P':0.,
        'G1P':0.,
        'SBP':0.,
        'S7P':0.,
        'E4P':0.,
        'X5P':0.,
        'R5P':0.,
        'RUBP':0.,
        'RU5P':.2,
        'Fluo':0.2,
        "Light":m.par.PFD}
init_ru5p=np.round(np.linspace(0.1,0.6,12),3)
rubspeicher=[]
res_list_atp=[]
res_list_starch=[]
res_list_vex=[]


fig = plt.figure()
ax1 = fig.add_subplot(111)




for i in init_ru5p:
    init_RU5P['RU5P']=i
    print(init_RU5P)
    y0 = np.array(list(init_RU5P.values()))
    s = modelbase.Simulator(m)
    s.set_initial_value(y0, t0=0)
    s.integrator.atol = 1.e-16
    s.model.par.update({'PFD':20})
    T=np.linspace(0.,600,300)
    s.timeCourse(T,y0)
    t=s.getT()
    res=s.getVarByName('RU5P')*1000


    ax1.plot(t,res,linewidth=4.,label=str(i))





ax1.plot(t,np.full(len(t),0.00248*1000),color='k',linewidth=3.,linestyle=':',label='Critical concentration')
ax1.set_ylabel(r'[RU5P] ($\mu$M)',fontsize=35.)
ax1.set_facecolor('white')
ax1.grid(color='lightgrey')
ax1.tick_params(axis='both', which='major', labelsize=30)
ax1.set_xlabel('Time (s)',fontsize=35.)
legend = ax1.legend(loc=0, ncol=1, bbox_to_anchor=(0, 0, 1, 1),
           prop={'size': 16},fancybox=True,shadow=False,title='Initial RU5P (mM)')

plt.setp(legend.get_title(),fontsize=20)
ax1.set_xlim((3,600))
ax1.set_ylim((0.,0.025*1000))
plt.title(str(20))
plt.show() 

The cell below includes Light-Dark-Light simulations, as previously performed, but with the additional Ru5P influx "oxPPP".

In [8]:
y0 = np.array(list(init.values()))
PhosphoMAX=15
fig = plt.figure()
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)


s = modelbase.Simulator(m)
s.model.par.update({'oxPPP' : 0.005})
s, m = LDL(s, y0, Tmax=2200,intensity=20.,darkphase=1000.,Tflash=100.)
t1=s.getT()

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex1=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub1=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='royalblue',linestyle=':',linewidth=4.,label='20 PFD',zorder=10.)
ax2.plot(t1,yvex1,color='royalblue',linestyle='--',linewidth=4.,label='20 PFD',zorder=10.)
ax3.plot(t1,yrub1,color='royalblue',linestyle='-.',linewidth=4.,label='20 PFD',zorder=10.)
sX1 = modelbase.Simulator(m)
sX1.model.par.update({'oxPPP' : 0.005})

s, m = LDL(sX1, y0, Tmax=2200,intensity=50.,darkphase=1000.,Tflash=100.)

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='forestgreen',linestyle=':',linewidth=4.,label='50 PFD',zorder=8.)
ax2.plot(t1,yvex2,color='forestgreen',linestyle='--',linewidth=4.,label='50 PFD',zorder=8.)
ax3.plot(t1,yrub2,color='forestgreen',linestyle='-.',linewidth=4.,label='50 PFD',zorder=8.)

sX1 = modelbase.Simulator(m)
sX1.model.par.update({'oxPPP' : 0.005})

s, m = LDL(sX1, y0, Tmax=2200,intensity=100.,darkphase=1000.,Tflash=100.)
t1=s.getT()

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='indianred',linestyle=':',linewidth=4.,label='100 PFD',zorder=6.)
ax2.plot(t1,yvex2,color='indianred',linestyle='--',linewidth=4.,label='100 PFD',zorder=6.)
ax3.plot(t1,yrub2,color='indianred',linestyle='-.',linewidth=4.,label='100 PFD',zorder=6.)

sX1 = modelbase.Simulator(m)
sX1.model.par.update({'oxPPP' : 0.005})

s, m = LDL(sX1, y0, Tmax=2200,intensity=150.,darkphase=1000.,Tflash=100.)
t1=s.getT()

y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='darkorange',linestyle=':',linewidth=4.,label='150 PFD',zorder=4.)
ax2.plot(t1,yvex2,color='darkorange',linestyle='--',linewidth=4.,label='150 PFD',zorder=4.)
ax3.plot(t1,yrub2,color='darkorange',linestyle='-.',linewidth=4.,label='150 PFD',zorder=4.)

sX1 = modelbase.Simulator(m)
sX1.model.par.update({'oxPPP' : 0.005})

s, m = LDL(sX1, y0, Tmax=2200,intensity=200.,darkphase=1000.,Tflash=100.)
t1=s.getT()
y1=s.getY()
YPi=s.getVarByName('Pi')
yvex2=s.getRate('vpga')+s.getRate('vgap')+s.getRate('vDHAP')
yrub2=s.getRate('vRuBisCO')

ax1.plot(t1,YPi/PhosphoMAX,color='k',linestyle=':',linewidth=4.,label='200 PFD',zorder=2.)
ax2.plot(t1,yvex2,color='k',linestyle='--',linewidth=4.,label='200 PFD',zorder=2.)
ax3.plot(t1,yrub2,color='k',linestyle='-.',linewidth=4.,label='200 PFD',zorder=2.)

ax1.axvspan(300,1300,alpha=0.4,color='dimgray')
ax2.axvspan(300,1300,alpha=0.4,color='dimgray')
ax3.axvspan(300,1300,alpha=0.4,color='dimgray')

ax1.set_facecolor('white')
ax1.grid(color='lightgrey')

ax2.set_facecolor('white')
ax2.grid(color='lightgrey')

ax3.set_facecolor('white')
ax3.grid(color='lightgrey')

ax1.set_ylabel('[Int. free phosphate] (mM)',fontsize=14.)
ax2.set_ylabel('TPT translocators (mM/s)',fontsize=14.)
ax3.set_ylabel('RuBisCO (mM/s)',fontsize=14.)

ax1.text(245,1.07, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax1.text(750, 1.07, 'D', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax1.text(1500, 1.07, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax1.set_ylim((0.4,1.15))


ax2.text(245,.25, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax2.text(750, .25, 'D', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax2.text(1500, .25, 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})

ax3.text(245,1., 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax3.text(750, 1., 'D', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})
ax3.text(1500, 1., 'L', fontsize=15,bbox={'facecolor':'w', 'alpha':0.5, 'pad':3})

ax2.legend(loc='best',bbox_to_anchor=(1., 0.7), prop={'size': 16})

ax1.tick_params(axis='both', which='major', labelsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax3.tick_params(axis='both', which='major', labelsize=18)

ax3.set_xlabel('Time (s)',fontsize=20.)

ax1.set_xlim((200,2200))
ax2.set_xlim((200,2200))
ax3.set_xlim((200,2200))

plt.show()


The following cell performs steady state simulations with the "oxPPP" RU5P influx reactions. The simulations are similar to the ones performed above in the code, but visualize different system properties.

In [17]:
PFDs=np.round(np.linspace(1.,300.,200),3)
rubspeicher=[]
res_list_atp=[]
res_list_starch=[]
res_list_vex=[]
for i in PFDs:
    
    print(i)
    y0 = np.array(list(init.values()))
    s = modelbase.Simulator(m)
    s.set_initial_value(y0, t0=0)
    s.integrator.atol = 1.e-16
    s.model.par.update({'oxPPP':0.})
    s.model.par.update({'PFD':i})
    T=np.linspace(0.,5e10)
    s.timeCourse(T,y0)
    
    rub=s.getRate('vRuBisCO')[-1]
    rubspeicher.append(rub)

    atpox=s.getVarByName('ATP')
    res_list_atp.append((atpox[-1]/2.55))
    starchox=s.getRate('vStarch')
    res_list_starch.append(starchox[-1])
    VEXox=(s.getRate('vpga')[-1]+s.getRate('vgap')[-1]+s.getRate('vDHAP')[-1])
    res_list_vex.append(VEXox)    
    
rubspeicher=np.array(rubspeicher)
res_list_atp=np.array(res_list_atp)
res_list_starch=np.array(res_list_starch)
res_list_vex=np.array(res_list_vex)
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

#plt.subplot(211)
ax1.plot(PFDs,rubspeicher,color='k',linewidth=5.5,label='RuBisCO')
ax1.plot(PFDs,res_list_starch,color='royalblue',linewidth=5.5,label='Starch prod.')
ax1.plot(PFDs,res_list_vex,color='indianred',linewidth=5.5,label='TPT export')



ax1.legend(loc='best')
ax1.set_ylabel('Flux in mM/s',fontsize=20.)
#plt.subplot(212)
ax2.plot(PFDs,res_list_atp,color='k',linewidth=5.5)
ax2.set_ylabel(r'$\frac{[ATP]}{[ATP]+[ADP]}$',fontsize=20.)
ax2.set_xlabel('Light intensity',fontsize=20.)

ax1.set_facecolor('white')
ax1.grid(color='lightgrey')
ax2.set_facecolor('white')
ax2.grid(color='lightgrey')

plt.show()

1.0
2.503
4.005
5.508
7.01
8.513
10.015
11.518
13.02
14.523
16.025
17.528
19.03
20.533
22.035
23.538
25.04
26.543
28.045
29.548
31.05
32.553
34.055
35.558
37.06
38.563
40.065
41.568
43.07
44.573
46.075
47.578
49.08
50.583
52.085
53.588
55.09
56.593
58.095
59.598
61.101
62.603
64.106
65.608
67.111
68.613
70.116
71.618
73.121
74.623
76.126
77.628
79.131
80.633
82.136
83.638
85.141
86.643
88.146
89.648
91.151
92.653
94.156
95.658
97.161
98.663
100.166
101.668
103.171
104.673
106.176
107.678
109.181
110.683
112.186
113.688
115.191
116.693
118.196
119.698
121.201
122.704
124.206
125.709
127.211
128.714
130.216
131.719
133.221
134.724
136.226
137.729
139.231
140.734
142.236
143.739
145.241
146.744
148.246
149.749
151.251
152.754
154.256
155.759
157.261
158.764
160.266


161.769
163.271
164.774
166.276
167.779
169.281
170.784
172.286
173.789
175.291
176.794
178.296
179.799
181.302
182.804
184.307
185.809
187.312
188.814
190.317
191.819
193.322
194.824
196.327
197.829
199.332
200.834
202.337
203.839
205.342
206.844
208.347
209.849
211.352
212.854
214.357
215.859
217.362
218.864
220.367
221.869
223.372
224.874
226.377
227.879
229.382
230.884
232.387
233.889
235.392
236.894
238.397
239.899
241.402
242.905
244.407
245.91
247.412
248.915
250.417
251.92
253.422
254.925
256.427
257.93
259.432
260.935
262.437
263.94
265.442
266.945
268.447
269.95
271.452
272.955
274.457
275.96
277.462
278.965
280.467
281.97
283.472
284.975
286.477
287.98
289.482
290.985
292.487
293.99
295.492
296.995
298.497
300.0
